Kernel : base (Python 3.9.7) conda environnement

# I. INSTALLER ENVIRONNEMENT DE TRAVAIL

Installer packages/librairies nécessaires

In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import os
from os import listdir
import missingno as msno
from zipfile import ZipFile
import requests
import math
%matplotlib inline
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, auc, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics.cluster import adjusted_rand_score

from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

from sklearn import (manifold, datasets, decomposition, ensemble,discriminant_analysis, random_projection)
from sklearn.metrics import silhouette_score
from matplotlib import offsetbox

import json
from pandas import json_normalize

Call request

In [2]:
import requests

url = "https://edamam-food-and-grocery-database.p.rapidapi.com/api/food-database/v2/parser"

#querystring = {"category[0]":"generic-foods","nutrition-type":"cooking","ingr":"champagn"}
querystring = {"nutrition-type":"cooking","ingr":"champagne"}

headers = {
	"X-RapidAPI-Key": "032ccf3819msh33cc0b8abc7ddd3p1320f9jsn6d62bccbcb81",
	"X-RapidAPI-Host": "edamam-food-and-grocery-database.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'text': 'champagne', 'parsed': [{'food': {'foodId': 'food_a656mk2a5dmqb2adiamu6beihduu', 'uri': 'http://www.edamam.com/ontologies/edamam.owl#Food_table_white_wine', 'label': 'Champagne', 'knownAs': 'dry white wine', 'nutrients': {'ENERC_KCAL': 82.0, 'PROCNT': 0.07, 'FAT': 0.0, 'CHOCDF': 2.6, 'FIBTG': 0.0}, 'category': 'Generic foods', 'categoryLabel': 'food', 'image': 'https://www.edamam.com/food-img/a71/a718cf3c52add522128929f1f324d2ab.jpg'}}], 'hints': [{'food': {'foodId': 'food_a656mk2a5dmqb2adiamu6beihduu', 'uri': 'http://www.edamam.com/ontologies/edamam.owl#Food_table_white_wine', 'label': 'Champagne', 'knownAs': 'dry white wine', 'nutrients': {'ENERC_KCAL': 82.0, 'PROCNT': 0.07, 'FAT': 0.0, 'CHOCDF': 2.6, 'FIBTG': 0.0}, 'category': 'Generic foods', 'categoryLabel': 'food', 'image': 'https://www.edamam.com/food-img/a71/a718cf3c52add522128929f1f324d2ab.jpg'}, 'measures': [{'uri': 'http://www.edamam.com/ontologies/edamam.owl#Measure_unit', 'label': 'Whole', 'weight': 750.0}, {'uri'

In [3]:
response.json().keys()

dict_keys(['text', 'parsed', 'hints', '_links'])

In [4]:
len(response.json()['parsed']), len(response.json()['hints']), len(response.json()['_links'])

(1, 20, 1)

In [5]:
top10 = response.json()['hints'][:10]

In [6]:
type(top10)

list

In [7]:
df = json_normalize(top10[0])

In [8]:
df

,measures,food.foodId,food.uri,food.label,food.knownAs,food.nutrients.ENERC_KCAL,food.nutrients.PROCNT,food.nutrients.FAT,food.nutrients.CHOCDF,food.nutrients.FIBTG,food.category,food.categoryLabel,food.image
0,[{'uri': 'http://www.edamam.com/ontologies/eda...,food_a656mk2a5dmqb2adiamu6beihduu,http://www.edamam.com/ontologies/edamam.owl#Fo...,Champagne,dry white wine,82.0,0.07,0.0,2.6,0.0,Generic foods,food,https://www.edamam.com/food-img/a71/a718cf3c52...


In [9]:
l = []
for i, j in enumerate(top10):
    df = json_normalize(top10[i])
    l.append(df)
df2 = pd.concat(l)

In [10]:
df3 = df2.reset_index(drop=True)

In [11]:
df3

,measures,food.foodId,food.uri,food.label,food.knownAs,food.nutrients.ENERC_KCAL,food.nutrients.PROCNT,food.nutrients.FAT,food.nutrients.CHOCDF,food.nutrients.FIBTG,food.category,food.categoryLabel,food.image,food.brand,food.foodContentsLabel,food.servingSizes,food.servingsPerContainer
0,[{'uri': 'http://www.edamam.com/ontologies/eda...,food_a656mk2a5dmqb2adiamu6beihduu,http://www.edamam.com/ontologies/edamam.owl#Fo...,Champagne,dry white wine,82.000000,0.070000,0.000000,2.600000,0.000000,Generic foods,food,https://www.edamam.com/food-img/a71/a718cf3c52...,NaN,NaN,NaN,NaN
1,[{'uri': 'http://www.edamam.com/ontologies/eda...,food_b753ithamdb8psbt0w2k9aquo06c,http://www.edamam.com/ontologies/edamam.owl#Fo...,"Champagne Vinaigrette, Champagne","CHAMPAGNE VINAIGRETTE, CHAMPAGNE",571.000000,0.000000,64.290001,7.140000,NaN,Packaged foods,food,NaN,SoFine Food,OLIVE OIL; BALSAMIC VINEGAR; CHAMPAGNE VINEGAR...,[{'uri': 'http://www.edamam.com/ontologies/eda...,NaN
2,[{'uri': 'http://www.edamam.com/ontologies/eda...,food_b3dyababjo54xobm6r8jzbghjgqe,http://www.edamam.com/ontologies/edamam.owl#Fo...,"Champagne Vinaigrette, Champagne","CHAMPAGNE VINAIGRETTE, CHAMPAGNE",333.000000,0.000000,36.669998,6.670000,NaN,Packaged foods,food,https://www.edamam.com/food-img/d88/d88b64d973...,Maple Grove Farms Of Vermont Inc.,INGREDIENTS: WATER; CANOLA OIL; CHAMPAGNE VINE...,[{'uri': 'http://www.edamam.com/ontologies/eda...,12.0
3,[{'uri': 'http://www.edamam.com/ontologies/eda...,food_a9e0ghsamvoc45bwa2ybsa3gken9,http://www.edamam.com/ontologies/edamam.owl#Fo...,"Champagne Vinaigrette, Champagne","CHAMPAGNE VINAIGRETTE, CHAMPAGNE",500.000000,0.000000,50.000000,6.670000,NaN,Packaged foods,food,NaN,T. Marzetti Company,CANOLA AND SOYBEAN OIL; WHITE WINE (CONTAINS S...,[{'uri': 'http://www.edamam.com/ontologies/eda...,NaN
4,[{'uri': 'http://www.edamam.com/ontologies/eda...,food_an4jjueaucpus2a3u1ni8auhe7q9,http://www.edamam.com/ontologies/edamam.owl#Fo...,"Champagne Vinaigrette, Champagne","CHAMPAGNE VINAIGRETTE, CHAMPAGNE",194.000000,0.000000,16.129999,6.450000,NaN,Packaged foods,food,NaN,T. Marzetti Company,WATER; CANOLA AND SOYBEAN OIL; WHITE WINE (CON...,[{'uri': 'http://www.edamam.com/ontologies/eda...,NaN
5,[{'uri': 'http://www.edamam.com/ontologies/eda...,food_bmu5dmkazwuvpaa5prh1daa8jxs0,http://www.edamam.com/ontologies/edamam.owl#Fo...,"Champagne Dressing, Champagne","CHAMPAGNE DRESSING, CHAMPAGNE",500.000000,0.000000,50.000000,6.670000,0.000000,Packaged foods,food,https://www.edamam.com/food-img/ab2/ab2459fc2a...,T. Marzetti Company,SOYBEAN OIL; WHITE WINE (PRESERVED WITH SULFIT...,[{'uri': 'http://www.edamam.com/ontologies/eda...,12.0
6,[{'uri': 'http://www.edamam.com/ontologies/eda...,food_alpl44taoyv11ra0lic1qa8xculi,http://www.edamam.com/ontologies/edamam.owl#_g...,Champagne Buttercream,Champagne Buttercream,431.284714,0.725971,21.529797,59.607171,0.000000,Generic meals,meal,NaN,NaN,sugar; butter; shortening; vanilla; champagne;...,NaN,NaN
7,[{'uri': 'http://www.edamam.com/ontologies/eda...,food_byap67hab6evc3a0f9w1oag3s0qf,http://www.edamam.com/ontologies/edamam.owl#_g...,Champagne Sorbet,Champagne Sorbet,116.246493,0.278922,0.641993,19.299747,0.181704,Generic meals,meal,NaN,NaN,Sugar; Lemon juice; brandy; Champagne; Peach,NaN,NaN
8,[{'uri': 'http://www.edamam.com/ontologies/eda...,food_am5egz6aq3fpjlaf8xpkdbc2asis,http://www.edamam.com/ontologies/edamam.owl#_g...,Champagne Truffles,Champagne Truffles,402.261444,6.599125,24.523124,48.494523,7.332186,Generic meals,meal,NaN,NaN,butter; cocoa; sweetened condensed milk; vanil...,NaN,NaN
9,[{'uri': 'http://www.edamam.com/ontologies/eda...,food_bcz8rhiajk1fuva0vkfmeakbouc0,http://www.edamam.com/ontologies/edamam.owl#_g...,Champagne Vinaigrette,Champagne Vinaigrette,412.109894,0.318194,42.780477,5.869073,0.422078,Generic meals,meal,NaN,NaN,champagne vinegar; olive oil; Dijon mustard; s...,NaN,NaN


In [12]:
to_keep = ['food.foodId', 'food.label', 'food.category', 'food.image', 'food.foodContentsLabel']

In [13]:
df4 = df3.drop(columns=[ele for ele in df2.columns if ele not in to_keep])

In [14]:
df4.head(3)

,food.foodId,food.label,food.category,food.image,food.foodContentsLabel
0,food_a656mk2a5dmqb2adiamu6beihduu,Champagne,Generic foods,https://www.edamam.com/food-img/a71/a718cf3c52...,NaN
1,food_b753ithamdb8psbt0w2k9aquo06c,"Champagne Vinaigrette, Champagne",Packaged foods,NaN,OLIVE OIL; BALSAMIC VINEGAR; CHAMPAGNE VINEGAR...
2,food_b3dyababjo54xobm6r8jzbghjgqe,"Champagne Vinaigrette, Champagne",Packaged foods,https://www.edamam.com/food-img/d88/d88b64d973...,INGREDIENTS: WATER; CANOLA OIL; CHAMPAGNE VINE...


In [15]:
df4.insert(3, 'food.foodContentsLabel', df4.pop('food.foodContentsLabel'))

In [16]:
df4.columns = ['foodId', 'label', 'category', 'foodContentsLabel', 'image']

In [17]:
df4.isna().sum().sum()

8

In [18]:
df4.to_csv('10st_products.csv', sep=';', index=False)